###  This notebook takes the calculated descriptors, retrains the model with all experimental data and estimates HAAFPs for all compounds in the PubChem database

In [ ]:
#conda install -c mordred-descriptor mordred

In [3]:
import pubchempy as pcp
import pandas as pd
import numpy as np
# import seaborn as sns

from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools

from mordred import Calculator, descriptors
from mordred import  ABCIndex

import matplotlib.pyplot as plt
from statistics import mean
from math import sqrt
from rdkit import Chem
from sklearn import metrics
from sklearn import model_selection
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

####  The sets of descriptors for all compounds are stored in a separate database - The data is read and formated in a single pandas dataframe

In [15]:
qsar_df1 = pd.read_csv("data/qsar_20201120_0_10000000.csv")
print("1")
qsar_df2 = pd.read_csv("data/qsar_20201120_10000000_20000000.csv")
print("2")
qsar_df3 = pd.read_csv("data/qsar_20201120_20000000_30000000.csv")
print("3")
qsar_df4 = pd.read_csv("data/qsar_20201120_30000000_40000000.csv")
qsar_df5 = pd.read_csv("data/qsar_20201120_40000000_50000000.csv")
print("5")
qsar_df6 = pd.read_csv("data/qsar_20201120_50000000_60000000.csv")
qsar_df7 = pd.read_csv("data/qsar_20201120_60000000_70000000.csv")
qsar_df8 = pd.read_csv("data/qsar_20201120_70000000_80000000.csv")
print("8")
qsar_df9 = pd.read_csv("data/qsar_20201120_80000000_90000000.csv")
qsar_df10 = pd.read_csv("data/qsar_20201120_90000000_100000000.csv")
print("10")
qsar_df11 = pd.read_csv("data/qsar_20201120_100000000_110000000.csv")
qsar_df12 = pd.read_csv("data/qsar_20201120_110000000_111456896.csv")
print("done")


done


In [ ]:
qsar_df = pd.concat([qsar_df1,qsar_df2,qsar_df3,qsar_df4,qsar_df5,qsar_df6,qsar_df7,qsar_df8,qsar_df9,qsar_df10,qsar_df11,qsar_df12], axis=0)
qsar_df.shape

In [ ]:
#### The "qsar" file contains all descriptors necessary for the prediction of　HAAs, here the descriptors are filtered according to the desired endpoint (DCAA or TCAA)

In [ ]:
features_list_TCAA_rf = ["SsOH",
                         "C2SP2",
                        "SlogP_VSA11",
                        "SMR_VSA7",
                        "PEOE_VSA1",
                        "EState_VSA7",
                        "ATSC5are",
                        "nBondsKD",
                        "ATS1i",
                        "nBondsM"]

features_list_DCAA_rf = ["AATSC0are",
                         "ATSC3dv",
                         "SlogP_VSA11",
                         "ATS4m",
                         "ATS4Z",
                         "Xch-7d",
                         "PEOE_VSA1",
                         "SaasC",
                         "piPC4",
                         "VSA_EState9"]
features_list_TCAA_DCAA_rf = ["SsOH",
                         "C2SP2",
                        "SlogP_VSA11",
                        "SMR_VSA7",
                        "PEOE_VSA1",
                        "EState_VSA7",
                        "ATSC5are",
                        "nBondsKD",
                        "ATS1i",
                        "nBondsM",
                        "AATSC0are",
                        "ATSC3dv",
                        "ATS4m",
                        "ATS4Z",
                        "Xch-7d",
                        "SaasC",
                        "piPC4",
                        "VSA_EState9" ] 

calc_dummy = Calculator(descriptors, ignore_3D=False)
my_desc_names_TCAA = features_list_TCAA_rf
my_desc_names_DCAA = features_list_DCAA_rf
my_desc_names_TCAA_DCAA = features_list_TCAA_DCAA_rf                        
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names_TCAA:
       my_descs.append(desc)

calc_TCAA = Calculator(my_descs, ignore_3D=False)
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names_DCAA:
       my_descs.append(desc)

calc_DCAA = Calculator(my_descs, ignore_3D=False)
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names_TCAA_DCAA:
       my_descs.append(desc)

calc_TCAA_DCAA = Calculator(my_descs, ignore_3D=False)

In [ ]:
calc=calc_TCAA_DCAA

In [ ]:
exp_descriptor_data = pd.DataFrame(calc.pandas(df_result['ROMol']))
X_Exp_TCAA = exp_descriptor_data[features_list_TCAA_rf]
X_Exp_DCAA = exp_descriptor_data[features_list_DCAA_rf]

#### Here the model is retrained with the experimental data

In [ ]:
mols = pd.read_excel("data/training_data.xlsx", engine = "openpyxl", header=0)
df_result = mols[['DCAA (umol/mmol)', 'TCAA (umol/mmol)', 'HAAFP (umol/mmol)']]
Y = df_result.values       ##at this point column 0 is DCAA and column 1 is TCAA
df_result = mols[['SMILES','DCAA (umol/mmol)', 'TCAA (umol/mmol)', 'HAAFP (umol/mmol)']]
PandasTools.AddMoleculeColumnToFrame(df_result, smilesCol='SMILES', includeFingerprints=False)
log_Y = np.log1p(Y)

In [ ]:
pipe_DCAA_rf = Pipeline(
    [
        ('std_scaler', StandardScaler()),
        ("RF", RandomForestRegressor(random_state = 17,
                                    n_estimators = 300,
                                    max_features = "auto",
                                    min_samples_split = 0.03,
                                    min_samples_leaf = 0.01,
                                    max_depth = 30,
                                    max_leaf_nodes = None,
                                    n_jobs = 1))
    ]
)

pipe_TCAA_rf = Pipeline(
    [
        ('std_scaler', StandardScaler()),
        ("RF", RandomForestRegressor(random_state = 17,
                                    n_estimators = 100,
                                    max_features = "auto",
                                     min_samples_split = 2,
                                     min_samples_leaf = 1,
                                     max_depth = None,
                                     max_leaf_nodes = None,
                                    n_jobs = 1))
    ]
)
pipe_TCAA_rf.fit(X_Exp_TCAA, log_Y[:,1])
pipe_DCAA_rf.fit(X_Exp_DCAA, log_Y[:,0]) #### Something wrong here!!!

#### qsar_df contains the necessary descriptors to predict HAAFPs.
#### Here qsar_df is passed on the trained model and new dataframes "predicted_DCAA_df" & "predicted_TCAA_df" are created and later concatenated to obtain the full results database "result_TCAA_DCAA_df", which is later exported as a csv file.   

In [18]:
qsar_df = pd.DataFrame(qsar_df, index = qsar_df.index).dropna(axis=0)

In [19]:
qsar_df.head()

,CID,ATS4Z,ATS4m,ATS1i,ATSC3dv,ATSC5are,AATSC0are,nBondsM,nBondsKD,C2SP2,Xch-7d,SaasC,SsOH,PEOE_VSA1,SMR_VSA7,SlogP_VSA11,EState_VSA7,VSA_EState9,piPC4
0,1,983.0,3451.906965,4531.610682,-46.196670,-2.196433,0.188575,2,2,0,0.000000,0.0,0.000000,19.120958,0.000000,0.0,21.143016,5.683855,3.091042
1,2,991.0,3466.046181,4716.794959,-48.277344,-2.335284,0.185105,2,2,0,0.000000,0.0,8.595382,14.326421,0.000000,0.0,21.143016,5.745079,3.091042
2,3,663.0,2323.102654,2822.099375,-0.969529,-0.268532,0.241330,3,3,3,0.175366,0.0,26.466667,15.319582,23.801165,0.0,0.000000,0.000000,3.761200
3,4,69.0,126.153216,2069.792191,-12.000000,0.384690,0.145978,0,0,0,0.000000,0.0,8.236111,10.840195,0.000000,0.0,0.000000,0.000000,0.000000
4,5,533.0,1911.181738,2673.307775,-47.035208,3.062675,0.323581,2,2,1,0.000000,0.0,16.115799,15.520491,0.000000,0.0,0.000000,-4.509159,2.197225


In [20]:
cid_df = qsar_df[["CID"]]
cid_df.shape

(111390281, 1)

In [ ]:
ext_TCAA_df = qsar_df[features_list_TCAA_rf]
ext_DCAA_df = qsar_df[features_list_DCAA_rf]
print("0")
Y_predicted_TCAA_rf = pipe_TCAA_rf.predict(ext_TCAA_df)
Y_predicted_DCAA_rf = pipe_DCAA_rf.predict(ext_DCAA_df)
print("1")
predicted_TCAA_df = pd.DataFrame(Y_predicted_TCAA_rf)
predicted_TCAA_df.columns = ["TCAA"]
print("2")
predicted_TCAA_df.index = qsar_df.index
predicted_DCAA_df = pd.DataFrame(Y_predicted_DCAA_rf)
predicted_DCAA_df.columns = ["DCAA"]
print("3")
predicted_DCAA_df.index = qsar_df.index
print("4")
result_TCAA_DCAA_df = pd.concat([cid_df,predicted_TCAA_df,predicted_DCAA_df], axis=1)
print("5")
# result_TCAA_DCAA_df = result_TCAA_DCAA_df.sort_values(by = ["TCAA"], ascending = False)
print("6")

0


In [ ]:
result_TCAA_DCAA_df.shape

In [ ]:
result_TCAA_DCAA_df_stored = result_TCAA_DCAA_df[["CID","TCAA","DCAA"]]

In [ ]:
result_TCAA_DCAA_df_stored.to_csv("/Users/andrescordero/Downloads/result_TCAA_DCAA_May12.csv",header=True,index=False)

#### A function showResult() is defined to show a sample of the results containing names, values and structures. 

In [ ]:
def showResult():
    result_TCAA_DCAA_df_10 = result_TCAA_DCAA_df.head(10)
    properties = ['IUPACName','CanonicalSMILES','MolecularFormula']
    cid_list = result_TCAA_DCAA_df_10["CID"].values.tolist()
    div_df = pd.DataFrame(pcp.get_properties(properties, cid_list, 'cid'))
    PandasTools.AddMoleculeColumnToFrame(div_df, smilesCol='CanonicalSMILES', includeFingerprints=False)
    result_TCAA_DCAA_df_10 = result_TCAA_DCAA_df_10.drop("CID", axis=1)
    result_TCAA_DCAA_df_10.index = div_df.index
    result_TCAA_DCAA_df_10 = pd.concat([div_df,result_TCAA_DCAA_df_10], axis=1)

In [ ]:
tcaa_distribution = result_TCAA_DCAA_df["TCAA"].values
# tcaa_distribution = np.exp(tcaa_distribution)-1
sns.displot(tcaa_distribution)

In [ ]:
result_TCAA_DCAA_df_10